# Reading and exploring NWB data from DANDI

This tutorial will demonstrate how to download, explore, and do basic visualizations with
an NWB File from DANDI.

**Our goals are to learn how to:**
- Download data from the [DANDI archive](https://gui.dandiarchive.org/)
- Read and explore an NWB file
- Do basic visualizations of NWB data
- Stream data from the [DANDI archive](https://gui.dandiarchive.org/)



## Import libraries

Before we get started, let's import a few libraries that we will use for accessing and visualizing the data. 

If you *are not* running this notebook on DANDI Hub, you will first need to install these packages using `pip` or your favorite Python package manager. For example:
```
pip install dandi pynwb remfile matplotlib
```

If you *are* running this notebook on DANDI Hub, all packages should be pre-installed except for `remfile`. We can install this package using the command below:

In [ ]:
!pip install remfile

In [ ]:
import h5py
import pathlib
import remfile
import matplotlib.pyplot as plt
import numpy as np

from dandi.download import download
from dandi.dandiapi import DandiAPIClient
from pynwb import NWBHDF5IO

# Downloading data from the DANDI archive

In this notebook we will be working with NWB data from one session of an experiment by
[Chandravadia et al. (2020)](https://www.nature.com/articles/s41597-020-0415-9). In this study,
the authors recorded single neuron activity from the medial temporal lobes of human subjects
while they performed a recognition memory task. This data can be found on the [DANDI Archive](https://gui.dandiarchive.org/) in [Dandiset 000004](https://gui.dandiarchive.org/dandiset/000004).


There are multiple ways to download data from DANDI:
1.  Downloading data using the DANDI Web UI
2.  Downloading data programmatically

## 1. Downloading data using the DANDI Web UI
You can download files directly from the DANDI website.

1. Go to dataset [000004](https://gui.dandiarchive.org/dandiset/000004) on the DANDI archive.
2. List the files in this dataset by clicking the "Files" button in Dandiset Actions (top right column of the page).

<img src="https://pynwb.readthedocs.io/en/latest/_images/demo_dandi_view_files_in_dataset.png" width="700" alt="view files on dandi" align="center">

3. Choose the folder "sub-P11MHM" by clicking on its name.

<img src="https://pynwb.readthedocs.io/en/latest/_images/demo_dandi_select_folder.png" width="700" alt="selecting a folder on dandi" align="center">

4. Download the NWB data file "sub-P11HMH_ses-20061101_ecephys+image.nwb" to your
computer by clicking on the download symbol.

<img src="https://pynwb.readthedocs.io/en/latest/_images/demo_dandi_download_data.png" width="700" alt="selecting a folder on dandi" align="center">




## 2. Downloading data programmatically
Alternatively, you can download data using the `DandiAPIClient` from the `dandi` Python package. Using the dandiset id and file path, we can use the dandi api to download the NWB file.

You can learn more about the dandi API in the [DANDI Python API docs](https://dandi.readthedocs.io/en/stable/modref/index.html).

In [ ]:
dandiset_id = '000004'
filepath = 'sub-P11HMH/sub-P11HMH_ses-20061101_ecephys+image.nwb'
with DandiAPIClient() as client:
    dandiset = client.get_dandiset(dandiset_id)
    asset = dandiset.get_asset_by_path(filepath)
    download(asset.download_url, '.')

In [ ]:
asset.download_url

# Reading and Exploring an NWB file

An [NWBFile](https://pynwb.readthedocs.io/en/latest/pynwb.file.html#pynwb.file.NWBFile) represents a single session of an experiment.
It contains all the data of that session and the metadata required to understand the data.


## Reading the NWB file
Reading and writing NWB data is carried out using the `NWBHDF5IO` class.
`NWBHDF5IO` reads and writes NWB data that is in the [HDF5](https://www.hdfgroup.org/solutions/hdf5/)
storage format, a popular, hierarchical format for storing large-scale scientific data.

### Using the NWBHDF5IO read method
The first argument to the constructor of `NWBHDF5IO` is the ``file_path``. Use the ``read`` method to read the data into a `NWBFile` object.

In [ ]:
# Open the file in read mode "r",
filepath = "sub-P11HMH_ses-20061101_ecephys+image.nwb"
io = NWBHDF5IO(filepath, mode="r")
nwbfile = io.read()

You can print the `NWBFile` object in a Jupyter notebook to get a simplified, interactive representation of the contents of the NWB file.

In [ ]:
nwbfile

### Using NWBHDF5IO as a context manager
`NWBHDF5IO` can also be used as a context manager.

The advantage of using a context manager is that the file is closed automatically when the context finishes
successfully or if there is an error. Be aware that if you use this method, closing the context (unindenting the code) will automatically close the `NWBHDF5IO` object and the corresponding h5py File object. The data not already read from the NWB file will then be inaccessible, so any code that reads data must be placed within the context.



In [ ]:
with NWBHDF5IO(filepath, mode="r") as io2:
    nwbfile2 = io2.read()

    # data accessible here

# data not accessible here

## Exploring the NWB file contents

### Accessing subject data

Access `nwbfile.subject` to get information about the subject used in this experiment, including their age, sex, species, and ID.

In [ ]:
nwbfile.subject

### Accessing stimulus data

Data representing stimuli that were presented to the experimental subject are stored in
`stimulus` within the `NWBFile` object.


In [ ]:
nwbfile.stimulus

``NWBFile.stimulus`` is a dictionary that can contain PyNWB objects representing
different types of data, such as images (grayscale, RGB) or time series of images.
In this file, ``NWBFile.stimulus`` contains a single key "StimulusPresentation" with an
`OpticalSeries` object representing what images were shown to the subject and at what times.



In [ ]:
stimulus_presentation = nwbfile.stimulus["StimulusPresentation"]
stimulus_presentation

## Lazy loading of datasets
Data arrays are read passively from the NWB file.
Accessing the `data` attribute of the `OpticalSeries` object does not read the data values, but presents an `h5py.Dataset` object that can be indexed to read data.
You can use the ``[:]`` operator to read the entire data array into memory.



In [ ]:
stimulus_presentation.data

In [ ]:
all_stimulus_data = stimulus_presentation.data[:]
all_stimulus_data

Images may be 3D or 4D (grayscale or RGB), where the first dimension must be time (frame).
The second and third dimensions represent x and y.
The fourth dimension represents the RGB value (length of 3) for color images.

This `OpticalSeries` data contains 200 images of size 400x300 pixels with three channels
(red, green, and blue).

In [ ]:
stimulus_presentation.data.shape

## Slicing datasets
Especially with very large datasets, it is often preferable to read only a portion of the data. To do this, index or slice into the ``data`` attribute just like if you were indexing or slicing a numpy array.

For example, let's look at a single image of the stimulus presentation data:


In [ ]:
frame_index = 31
image = stimulus_presentation.data[frame_index]
image = image[..., ::-1] # Reverse the last dimension because the data were stored in BGR instead of RGB
plt.imshow(image, aspect="auto")

# Visualizing NWB Data

## Accessing single unit data
Data and metadata about sorted single units are stored in a `Units`
object. It stores metadata about each single unit in a tabular form, where each row represents
a unit with spike times and additional metadata.

In [ ]:
nwbfile.units

We can view the single unit data as a pandas `DataFrame`.



In [ ]:
units_df = nwbfile.units.to_dataframe()
units_df.head()

To access the spike times of the first single unit, index this pandas dataframe with the column name, “spike_times”, and the row index, 0. All times in NWB are stored in seconds relative to the session start time.

In [ ]:
units_df["spike_times"][0]

## Visualizing spiking activity relative to stimulus onset
We can look at when these single units spike relative to when image stimuli were presented to the subject.
We will iterate over the first 3 units and get their spike times.
Then for each unit, we will iterate over each stimulus onset time and compute the spike times relative
to stimulus onset. Finally, we will create a raster plot and histogram of these aligned spike times.



In [ ]:
before = 1.0  # in seconds
after = 3.0

# Get the stimulus times for all stimuli
# get_timestamps() works whether the time is stored as an array of timestamps or as
# starting time and sampling rate.
stim_on_times = stimulus_presentation.get_timestamps()

for unit in range(3):
    unit_spike_times = nwbfile.units["spike_times"][unit]
    trial_spikes = []
    for time in stim_on_times:
        # Compute spike times relative to stimulus onset
        aligned_spikes = unit_spike_times - time
        # Keep only spike times in a given time window around the stimulus onset
        aligned_spikes = aligned_spikes[
            (-before < aligned_spikes) & (aligned_spikes < after)
        ]
        trial_spikes.append(aligned_spikes)
    fig, axs = plt.subplots(2, 1, sharex="all")
    plt.xlabel("time (s)")
    axs[0].eventplot(trial_spikes)

    axs[0].set_ylabel("trial")
    axs[0].set_title("unit {}".format(unit))
    axs[0].axvline(0, color=[0.5, 0.5, 0.5], linestyle='dashed')

    axs[1].hist(np.hstack(trial_spikes), 30)
    axs[1].axvline(0, color=[0.5, 0.5, 0.5], linestyle='dashed')

## Accessing trials
Trials are stored as `TimeIntervals` object which is a subclass of `DynamicTable`. `DynamicTable` objects are used to store metadata about each trial in a tabular form, where each row represents a trial and has a start time, stop time, and additional metadata.

You can find more information about trials and time intervals in the [pynwb time intervals Tutorial](https://pynwb.readthedocs.io/en/latest/tutorials/general/plot_timeintervals.html#time-intervals)

Similarly to `Units`, we can view trials as a pandas `DataFrame`.


In [ ]:
trials_df = nwbfile.trials.to_dataframe()
trials_df.head()

## Visualizing stimulus images by trial
The stimulus can be mapped one-to-one to each row (trial) of `trials` based on the ``stim_on_time`` column.



In [ ]:
assert np.all(stimulus_presentation.timestamps[:] == trials_df.stim_on_time[:])

Here we will visualize the first 3 images that were categorized as landscapes in the session.

In [ ]:
stim_on_times_landscapes = trials_df[
    trials_df.category_name == "landscapes"
].stim_on_time
for time in stim_on_times_landscapes.iloc[:3]:
    img = np.squeeze(
        stimulus_presentation.data[
            np.where(stimulus_presentation.timestamps[:] == time)
        ]
    )
    # Reverse the last dimension because the data were stored in BGR instead of RGB
    img = img[..., ::-1]
    plt.figure()
    plt.imshow(img, aspect="auto")

## Further exploration and visualization tools
So far we have explored the NWB file by inspecting at the `NWBFile` object and accessing its attributes, but it may be useful to explore the data in a
more interactive, visual way. See [Exploring NWB Files](https://nwb-overview.readthedocs.io/en/latest/tools/analysis_tools_home.html#analysistools-explore) in the analysis tool catalog for an updated list of programs for exploring NWB files.

# Streaming data

In some circumstances, you may want to work with very large NWB files or with subsets of data from an NWB file. Instead of downloading the full NWB file, another approach is to stream data directly from an archive. Streaming data allows you to read specific sections within individual data files directly from remote stores such as
[DANDI](https://dandiarchive.org/).

There are multiple methods to stream NWB files, we will focus on using `remfile` today but you can find additional options in the [pynwb streaming tutorial](https://pynwb.readthedocs.io/en/stable/tutorials/advanced_io/streaming.html)

## Getting the location of the file on DANDI
First, you will use the `DandiAPIClient` get the S3 URL of the NWB file stored on DANDI. Similarly to downloading the file, we can use the dandiset ID and the path of the file within that dataset to get this information.

In [ ]:
dandiset_id = '000004'
filepath = 'sub-P11HMH/sub-P11HMH_ses-20061101_ecephys+image.nwb'
with DandiAPIClient() as client:
    asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(filepath)
    s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)

In [ ]:
s3_url

## Using remfile to stream data
`remfile` is a library that enables indexing and streaming of files in s3. `remfile` is simple and fast, especially for the initial load of the nwb file and for accessing small pieces of data. The caveats of `remfile` are that it is a new project that has not been tested in a variety of use-cases and caching options are limited compared to other methods such as `fsspec`.

In [ ]:
rem_file = remfile.File(s3_url)

file = h5py.File(rem_file, "r")
io_stream = NWBHDF5IO(file=file)
nwbfile_stream = io_stream.read()

You can see how we can still get a simplified interactive representation of the nwb file contents after streaming.

In [ ]:
nwbfile_stream

We can work with the data in the same way as downloading the full file, but now we will stream the data on demand as we need it.

In [ ]:
frame_index = 31
image = nwbfile_stream.stimulus['StimulusPresentation'].data[frame_index]
image = image[..., ::-1] # Reverse the last dimension because the data were stored in BGR instead of RGB
plt.imshow(image, aspect="auto")

## Closing the open NWB files
It is good practice to close any files that you have opened.



In [ ]:
io.close()         # downloaded io object
io_stream.close()  # streaming io object